In [1]:
import os
import pickle


with open('Models/AdaBoost_model.pkl', 'rb') as file:
    AdaBoost_model = pickle.load(file)
with open('Models/HGB_model.pkl', 'rb') as file:
    HGB_model = pickle.load(file)
with open('Models/ExtraTrees_model.pkl', 'rb') as file:
    ExtraTrees_model = pickle.load(file)
with open('Models/RF_undersample.pkl', 'rb') as file:
    RandomForest_model = pickle.load(file)
with open('Models/SVM_Oversample_model.pkl', 'rb') as file:
    SVM_model = pickle.load(file)
with open('Models/LR_Oversampled_model.pkl', 'rb') as file:
    LogisticRegression_model = pickle.load(file)



In [2]:
import pandas as pd

data=pd.read_csv('Data/colon-dataset-processed.csv')

#split the data into X and y
X=data.drop('Class',axis=1)
y=data['Class']

#split the data into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state = 42)

In [10]:
pred1=AdaBoost_model.predict(X_test)
pred2=HGB_model.predict(X_test)
pred3=ExtraTrees_model.predict(X_test)
pred4=RandomForest_model.predict(X_test)
pred5=SVM_model.predict(X_test)

print(type(pred1),type(pred2),type(pred3),type(pred4),type(pred5))

pred_final=pred1+pred2+pred3+pred4+pred5/5.0



<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


TypeError: can only concatenate str (not "int") to str

In [4]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, pred_final)
accuracy

0.8571428571428571

In [5]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,ensemble_predictions)
cm

array([[15,  3],
       [ 2, 15]])

<h1>Stacking</h1>

In [6]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression()

stack_models=[
    ('SVM', SVM_model),
    ('RF', RandomForest_model),
    ('ET', ExtraTrees_model),
    ('HGB', HGB_model),
    ('AB', AdaBoost_model)
]

stacking_model = StackingClassifier(estimators=stack_models, final_estimator=meta_model)

stacking_model.fit(X_train, y_train)

predictions = stacking_model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
accuracy

0.8571428571428571